In [1]:
!pip install torchxrayvision==1.2.1 torchvision
!pip install torch
!pip install scikit-learn
!pip install tqdm
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import os
os.listdir("/kaggle/input/chexpert")


['valid.csv', 'valid', 'train.csv', 'train']

In [3]:
!ln -s /kaggle/input/chexpert /kaggle/working/CheXpert-v1.0-small

In [4]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score

In [5]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

CHEXPERT14 = [
    "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity",
    "Lung Lesion", "Edema", "Consolidation", "Atelectasis",
    "Pneumothorax", "Pleural Effusion", "Support Devices"
]

class CheXpertDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        if "Path" not in self.df.columns and "path" in self.df.columns:
            self.df.rename(columns={"path": "Path"}, inplace=True)

        drop_cols = ["Sex", "Age", "Frontal/Lateral", "AP/PA", "View", "Unnamed: 0"]
        self.df = self.df.drop(columns=[c for c in drop_cols if c in self.df.columns], errors="ignore")

        present_labels = [c for c in CHEXPERT14 if c in self.df.columns]
        self.df = self.df[["Path"] + present_labels].copy()
        self.label_cols = present_labels

        for c in self.label_cols:
            self.df[c] = pd.to_numeric(self.df[c], errors="coerce")  # 非法转成 NaN
            self.df[c] = self.df[c].replace(-1, 0)                   # -1 -> 0
            self.df[c] = self.df[c].fillna(0).astype(np.float32)     # NaN -> 0, 并转 float32


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["Path"].lstrip("/"))

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        labels_np = row[self.label_cols].to_numpy(dtype=np.float32, copy=True)
        labels = torch.from_numpy(labels_np)  # dtype=float32
        return image, labels


In [6]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = CheXpertDataset(
    csv_path="/kaggle/input/chexpert/train.csv",
    img_dir="/kaggle/working",
    transform=train_transform
)
val_dataset = CheXpertDataset(
    csv_path="/kaggle/input/chexpert/valid.csv",
    img_dir="/kaggle/working",
    transform=val_transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(weights='IMAGENET1K_V1')  

num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 10)  
model = model.to(device)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 211MB/s]


In [8]:
criterion = nn.BCEWithLogitsLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [9]:
best_auc = 0.0
EPOCHS = 3

for epoch in range(EPOCHS):

    # ---------------- TRAINING ----------------
    model.train()
    running_loss = 0.0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss = 0
    preds_all, labels_all = [], []

    from sklearn.metrics import roc_auc_score
    import numpy as np

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc="Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.sigmoid(outputs).cpu().numpy()
            preds_all.append(preds)
            labels_all.append(labels.cpu().numpy())

    # Combine batch results
    avg_val_loss = val_loss / len(val_loader)
    preds_all = np.concatenate(preds_all, axis=0)
    labels_all = np.concatenate(labels_all, axis=0)

    # ---------------- COMPUTE AUC ----------------
    valid_aucs = []
    for i in range(labels_all.shape[1]):
        y_true = labels_all[:, i]
        y_pred = preds_all[:, i]

        if len(np.unique(y_true)) < 2:  
            print(f"Label {i} has only one class in validation set — skipping AUC.")
            continue

        auc_i = roc_auc_score(y_true, y_pred)
        valid_aucs.append(auc_i)

    if len(valid_aucs) > 0:
        val_auc = np.mean(valid_aucs)
    else:
        val_auc = np.nan
        print("All labels had only one class — AUC cannot be computed.")

    print(f"Validation: Loss = {avg_val_loss:.4f} | AUC = {val_auc:.4f}")

    # ---------------- SAVE MODEL ----------------
    # Save model for each epoch
    model_path = f"model_epoch_{epoch+1}_auc_{val_auc:.4f}.pt"
    torch.save(model.state_dict(), model_path)
    print(f" Model saved: {model_path}")

    # Save best model
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), "best_densenet_chexpert.pt")
        print(f" Best model updated — AUC improved to {best_auc:.4f}")

    # Learning rate scheduler
    scheduler.step()

Epoch 1/3: 100%|██████████| 13964/13964 [1:56:16<00:00,  2.00it/s]



Epoch 1: Train Loss = 0.3500


Validation: 100%|██████████| 15/15 [00:06<00:00,  2.41it/s]


Validation: Loss = 0.4957 | AUC = 0.7905
 Model saved: model_epoch_1_auc_0.7905.pt
 Best model updated — AUC improved to 0.7905


Epoch 2/3: 100%|██████████| 13964/13964 [56:40<00:00,  4.11it/s]


...

Validation: Loss = 0.4821 | AUC = 0.8319
 Model saved: model_epoch_2_auc_0.8319.pt
 Best model updated — AUC improved to 0.8319


Epoch 3/3: 100%|██████████| 13964/13964 [54:58<00:00,  4.23it/s]



Epoch 3: Train Loss = 0.3242


Validation: 100%|██████████| 15/15 [00:02<00:00,  6.32it/s]


Validation: Loss = 0.5024 | AUC = 0.8366
 Model saved: model_epoch_3_auc_0.8366.pt
 Best model updated — AUC improved to 0.8366


In [10]:
import os, shutil

os.makedirs("/kaggle/outputs", exist_ok=True)

for file in os.listdir("/kaggle/working"):
    if file.endswith(".pt"):
        src = os.path.join("/kaggle/working", file)
        dst = os.path.join("/kaggle/outputs", file)
        shutil.copy(src, dst)
        print(f"{file}")

print("\n All models have been successfully copied to /kaggle/outputs. You can download them from the Output → Files page.")

model_epoch_3_auc_0.8366.pt
model_epoch_2_auc_0.8319.pt
model_epoch_1_auc_0.7905.pt
best_densenet_chexpert.pt

 All models have been successfully copied to /kaggle/outputs. You can download them from the Output → Files page.


In [11]:
!ls -lh /kaggle/outputs

total 109M
-rw-r--r-- 1 root root 28M Nov 15 08:54 best_densenet_chexpert.pt
-rw-r--r-- 1 root root 28M Nov 15 08:54 model_epoch_1_auc_0.7905.pt
-rw-r--r-- 1 root root 28M Nov 15 08:54 model_epoch_2_auc_0.8319.pt
-rw-r--r-- 1 root root 28M Nov 15 08:54 model_epoch_3_auc_0.8366.pt


In [12]:
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

ckpt_path = "best_densenet_chexpert.pt"  
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()

val_loss = 0.0
preds_all, labels_all = [], []

with torch.no_grad():
    for imgs, labels in tqdm(val_loader, desc="Validation"):
        imgs = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs)                      
        loss = criterion(logits, labels)         
        val_loss += loss.item()

        probs = torch.sigmoid(logits).cpu().numpy()   
        preds_all.append(probs)
        labels_all.append(labels.cpu().numpy())

avg_val_loss = val_loss / len(val_loader)
preds_all = np.concatenate(preds_all, axis=0)   # [N, C]
labels_all = np.concatenate(labels_all, axis=0) # [N, C]

valid_aucs = []
skipped = []

num_classes = labels_all.shape[1]
for i in range(num_classes):
    y_true = labels_all[:, i]
    y_pred = preds_all[:, i]
    if len(np.unique(y_true)) < 2:
        skipped.append(i)
        continue
    auc_i = roc_auc_score(y_true, y_pred)
    valid_aucs.append(auc_i)

val_auc = np.mean(valid_aucs) if len(valid_aucs) > 0 else np.nan

print("========== Validation Summary ==========")
print(f"Loss: {avg_val_loss:.4f}")
print(f"Macro AUC (valid cols): {val_auc:.4f}" if not np.isnan(val_auc) else "Macro AUC: NaN")
if skipped:
    print(f"Skipped label indices (no 0/1 mix in val): {skipped}")
print(f"Used {len(valid_aucs)}/{num_classes} labels for AUC.")


Validation: 100%|██████████| 15/15 [00:01<00:00,  8.31it/s]

========== Validation Summary ==========
Loss: 0.5024
Macro AUC (valid cols): 0.8366
Used 10/10 labels for AUC.


In [13]:
label_names = train_dataset.label_cols   # the 10 CheXLocalize labels

per_label_auc = {}

for i, label in enumerate(label_names):
    y_true = labels_all[:, i]
    y_pred = preds_all[:, i]

    # Skip labels with only one class in validation (all 0 or all 1)
    if len(np.unique(y_true)) < 2:
        per_label_auc[label] = "N/A (only one class present)"
        continue

    auc_i = roc_auc_score(y_true, y_pred)
    per_label_auc[label] = auc_i

print("\n========== PER-LABEL AUC ==========\n")
for label, auc in per_label_auc.items():
    print(f"{label:30} : {auc}")

# Mean AUC over valid labels
valid_aucs = [v for v in per_label_auc.values() if isinstance(v, float)]
print("\nMacro AUC:", np.mean(valid_aucs))


========== PER-LABEL AUC ==========

Enlarged Cardiomediastinum     : 0.5037798165137615
Cardiomegaly                   : 0.8067859673990078
Lung Opacity                   : 0.9099059376837154
Lung Lesion                    : 0.9570815450643777
Edema                          : 0.9247501469723692
Consolidation                  : 0.9047188300919644
Atelectasis                    : 0.7525162337662339
Pneumothorax                   : 0.7782079646017699
Pleural Effusion               : 0.9253731343283582
Support Devices                : 0.9028626094635367

Macro AUC: 0.8365982185885095
